# BDSチュートリアル

In [1]:
import pandas as pd

In [2]:
from wrappers.bds_stan_wrapper import BDS

In [3]:
from crowdkit.aggregation import DawidSkene
from crowdkit.datasets import load_dataset

## Load Data
サンプルデータとして，以下のデータをダウンロードして，`sample_dataset.csv`として保存してください．

https://github.com/crowd4u/crowd_aggregation_datasets/blob/main/TinyImagenet/dataset.csv

上記ファイルでなくても，`crowd-kit`形式ならなんでもいいです．

In [4]:
df = pd.read_csv("sample_dataset.csv")
gt = df.filter(["task","gt"]).drop_duplicates(keep='last').set_index("task")
df = df.drop(["gt"], axis=1)

In [5]:
df.head()

,task,worker,label
0,0-48,A3KAF6CU2BYVLG,0
1,0-48,A361ONJ6UNYLSU,0
2,3-82,A3PK30QCJZFMBP,2
3,3-82,A3PND2XNVWO4JF,2
4,2-474,A2WMX6GLWV0KC,3


**注意**：`label`が0から始まる数字になっていない場合は，事前に変換処理が必要です．

## Run BDS
`algorithm="mcmc" or "vb"`が選べますが，MCMCの方がおすすめです．

In [6]:
# 初期値の設定
infer_params = {
    "iter_warmup" : 500,
    "iter_sampling" : 1000,
    # ここに指定できるパラメータは・・・
    # MCMCの場合は以下のドキュメントを参照
    # https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanModel.sample
    # 変分ベイズの場合は
    # https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanModel.variational
}

init_worker_accuracy = 0.75
# ワーカーの事前分布に対角成分が0.75になるフラットな混同行列を仮定する

labels = [0,1,2,3,4] # データセットに含まれるラベルを指定

In [7]:
bds = BDS(labels, infer_params=infer_params, init_worker_accuracy=init_worker_accuracy, algorithm="mcmc")

09:41:08 - cmdstanpy - INFO - compiling stan file /app/stan_models/DS.stan to exe file /app/stan_models/DS
09:41:39 - cmdstanpy - INFO - compiled model executable: /app/stan_models/DS
09:41:39 - cmdstanpy - WARNING - Stan compiler has produced 8 warnings:
09:41:39 - cmdstanpy - WARNING - 
--- Translating Stan model to C++ code ---
bin/stanc --filename-in-msg=DS.stan --o=/app/stan_models/DS.hpp /app/stan_models/DS.stan
Warning in 'DS.stan', line 6, column 2: Declaration of arrays by placing
    brackets after a variable name is deprecated and will be removed in Stan
    2.33.0. Instead use the array keyword before the type. This can be
    changed automatically using the auto-format flag to stanc
Warning in 'DS.stan', line 7, column 2: Declaration of arrays by placing
    brackets after a variable name is deprecated and will be removed in Stan
    2.33.0. Instead use the array keyword before the type. This can be
    changed automatically using the auto-format flag to stanc
Warning in '

In [8]:
ret = bds.fit_predict(df)

09:42:40 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

09:47:33 - cmdstanpy - INFO - CmdStan done processing.



Checking Rhat values
pi (r_hat > 1.1): 0
q_z:
	Class 0 (r_hat > 1.1): 0
	Class 1 (r_hat > 1.1): 0
	Class 2 (r_hat > 1.1): 0
	Class 3 (r_hat > 1.1): 0
	Class 4 (r_hat > 1.1): 0


$\hat{R}$はMCMCの収束性を判定することに利用できる値で，各変数ごとに計算されます．$\hat{R}>1.1$であれば収束しているとみなされます．上記のサマリーでは，DS法の各変数ごとに収束していない変数の数を表示しています．

In [9]:
ret.head()

task
0-48     0
3-82     2
2-474    3
4-356    4
2-247    2
Name: label, dtype: int64

`bds_stan_wrapper.py`は僕が書いたコードなので，頑張って解読して後は頑張ってください．